# import the libraries

In [398]:
import ee
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Authentication and Initialization
  -Before you can make requests to Earth Engine through a client library, you must authenticate

In [400]:
ee.Authenticate()

True

In [401]:
ee.Initialize()

# Get statistics of Rectangels(regions) around the specific points

In [403]:
#dowenload the image collection and filter it with date and get the median image
dataset = ee.ImageCollection('MODIS/061/MOD09A1').filterDate('2022-09-01', '2022-09-30').median()  

In [404]:
# Define the points (coordinates)
points = [
    [30.488472, 31.173472],
    [30.524722, 31.151861],
    [30.566028, 31.130972],
    [30.647556, 31.077806],
    [30.734028, 31.028500],
    [30.807556, 31.013361],
    [30.790639, 30.965194],
    [30.801278, 30.913278],
    [30.815306, 30.904389],
    [30.848333, 30.901222],
    [30.876917, 30.873500],
    [30.899056, 30.856250],
    [30.930333, 30.835639],
    [30.961583, 30.783083],
    [31.003111, 30.759333],
    [31.043556, 30.735694],
    [31.085667, 30.727861],
    [31.101222, 30.683639],
    [31.143611, 30.645361],
    [31.182500, 30.600361],
    [31.207500,30.569444],
    [31.224444,30.531583],
    [31.264361,30.518389],
    [31.311250,30.515528],
    [31.344417,30.482861],
    [30.851444,31.030083],
    [30.894389,31.024361],
    [30.893667,31.024667],
    [30.931611,30.990083],
    [30.980056,30.970417],
    [31.031639,30.961556],
    [31.085750,30.965417],
    [31.142111,30.968861],
    [31.214722,30.984111],
    [31.268806,30.990528],
    [31.261778,31.049611],
    [31.278194,31.093417],
    [31.328944,31.132139],
    [31.399000,31.125000],
    [31.471750,31.126861],
    [31.526444,31.140306],
    [31.565361,31.081889],
    [31.586472,31.052500],
    [31.412167,30.451000],
    [31.340417,30.366278],
    [31.205000,30.091444],
    [31.167278,30.069167],
    [31.128167,30.102194],
    [31.120861,30.164528],
    [31.108250,30.211944],
    [31.086417,30.294417],
    [31.069194,30.359806],
    [31.013250,30.473583],
]

In [405]:
# Create a feature collection from the points
features = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(coord)) for coord in points])

In [406]:
# Define the size of the rectangle in meters (1 km x 1 km)
size = 7000  # Size in meters

In [407]:
# Create a function to create a rectangle around a point
def create_rectangle(point):
    lat = point[1]
    lon = point[0]
    
    # Calculate the corners of the rectangle
    topLeft = [
        lon - (size / 20037508.34 * 180 / 3.141592653589793),
        lat + (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    bottomRight = [
        lon + (size / 20037508.34 * 180 / 3.141592653589793),
        lat - (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    # Return the rectangle geometry
    return ee.Geometry.Rectangle(
        topLeft[0], bottomRight[1],
        bottomRight[0], topLeft[1]
    )

In [408]:
# Create a feature collection of rectangles
rectangles = ee.FeatureCollection([ee.Feature(create_rectangle(pt)) for pt in points])

In [409]:
# Function to get statistics for each rectangel
def get_statistics(feature):
    point = feature.geometry()
    # Calculate the median for each band in the median image
    stats = dataset.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=point,
        scale=30  # resolution in meters
    )
    return feature.set(stats)

In [410]:
# Apply the function to each rectangel in the features collection
stats_features = rectangles.map(get_statistics)

In [437]:
# Extract features and their statistics from server-side
features = stats_features.getInfo().get('features', [])

# prepare the tabel data to store the statistics results

In [411]:
# Initialize a data table as a dictionary
data_table = {
    'Point Longitude': [],
    'Point Latitude': [],
    'sur_refl_b01': [],
    'sur_refl_b02': [],
    'sur_refl_b03': [],
    'sur_refl_b04': [],
    'sur_refl_b05': [],
    'sur_refl_b06': [],
    'sur_refl_b07': []
}
# Loop through features and populate the data table
for feature in features:
    properties = feature['properties']
    coords = feature['geometry']['coordinates'][0]  # Assuming the coordinates are in the first list
    
    # Calculate the centroid manually since Python does not have direct access like JS
    centroid_lon = sum([coord[0] for coord in coords]) / len(coords)
    centroid_lat = sum([coord[1] for coord in coords]) / len(coords)

    # Populate data_table with the centroid coordinates and properties
    data_table['Point Longitude'].append(centroid_lon)
    data_table['Point Latitude'].append(centroid_lat)
    data_table['sur_refl_b01'].append(properties.get('sur_refl_b01'))
    data_table['sur_refl_b02'].append(properties.get('sur_refl_b02'))
    data_table['sur_refl_b03'].append(properties.get('sur_refl_b03'))
    data_table['sur_refl_b04'].append(properties.get('sur_refl_b04'))
    data_table['sur_refl_b05'].append(properties.get('sur_refl_b05'))
    data_table['sur_refl_b06'].append(properties.get('sur_refl_b06'))
    data_table['sur_refl_b07'].append(properties.get('sur_refl_b07'))

In [413]:
#Create a DataFrame to display statistics
df = pd.DataFrame(data_table)

# Print the DataFrame
print(df)

# Save the result to a CSV file
csv_filename = 'modisrec_statistics.csv'
df.to_csv(csv_filename, index=False)
print(f'Results saved to {csv_filename}')

    Point Longitude  Point Latitude  sur_refl_b01  sur_refl_b02  sur_refl_b03  \
0         30.484469       31.169469    831.000000   3268.763022    466.000000   
1         30.520719       31.147858    891.000000   3063.000000    504.000000   
2         30.562025       31.126969    915.000000   3070.155882    496.000000   
3         30.643553       31.073803    880.000000   2555.425976    465.000000   
4         30.730025       31.024497    796.996303   2726.377377    449.651387   
5         30.803553       31.009358    848.969589   2777.862023    462.337137   
6         30.786636       30.961191    816.910820   2328.998152    442.000000   
7         30.797275       30.909275    754.001233   2362.500000    426.000000   
8         30.811303       30.900386    753.094118   2373.133603    426.000000   
9         30.844330       30.897219    824.895651   2368.000000    444.584311   
10        30.872914       30.869497    853.515397   2404.479913    456.175881   
11        30.895053       30

# calculate the correlation between bands and HM

In [415]:
# read data and display it 
data=pd.read_csv('modisrec_statistics.csv')
data.head()

,Point Longitude,Point Latitude,sur_refl_b01,sur_refl_b02,sur_refl_b03,sur_refl_b04,sur_refl_b05,sur_refl_b06,sur_refl_b07
0,30.484469,31.169469,831.000000,3268.763022,466.000000,830.667076,2756.392387,1957.629052,1093.658080
1,30.520719,31.147858,891.000000,3063.000000,504.000000,879.000000,2733.000000,2042.085014,1240.000000
2,30.562025,31.126969,915.000000,3070.155882,496.000000,879.000000,2748.000000,2084.508772,1236.514286
3,30.643553,31.073803,880.000000,2555.425976,465.000000,807.000000,2367.000000,1866.000000,1167.000000
4,30.730025,31.024497,796.996303,2726.377377,449.651387,833.674306,2421.348214,1653.242381,936.344035


In [416]:
# add the HM concentration column to df
data['HM']=[
384.9,
318.3,
342.4,
440.8,
397.6,
380.4,
381.2,
414.5,
333.9,
362.8,
366.9,
407.6,
373.5,
382.0,
385.6,
356.8,
293.4,
332.1,
363.2,
429.9,
378.6,
419.2,
399.6,
189.0,
427.1,
429.2,
392.4,
387.6,
437.0,
444.5,
410.6,
453.4,
409.2,
448.0,
376.7,
395.0,
397.7,
347.8,
328.7,
292.1,
222.7,
152.0,
141.5,
287.4,
340.4,
318.1,
309.6,
325.8,
246.8,
231.3,
326.1,
276.7,
340.9]

In [417]:
# split the target(HM) and get correlation between it and features(bands)
target='HM'
#pearson correlation
corr=data.corr()[target]
print(corr)

Point Longitude   -0.457153
Point Latitude     0.234361
sur_refl_b01      -0.303686
sur_refl_b02      -0.204857
sur_refl_b03      -0.262031
sur_refl_b04      -0.332050
sur_refl_b05      -0.333678
sur_refl_b06      -0.478731
sur_refl_b07      -0.347072
HM                 1.000000
Name: HM, dtype: float64
